In [3]:
from pathlib import Path
from typing import List

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import amazon_dataset

# Intialize Datasets

In [4]:
#DATASET = 'Musical_Instruments'
#DATASET = 'Clothing_Shoes_and_Jewelry'
#DATASET = 'Home_and_Kitchen'
DATASET = 'Movies_and_TV'

VALIDATION_SIZE=0.15
RANDOM_SEED = 20230219
#ALEXNET_IMAGE_FEATURES = Path(f'data/amazon/{DATASET}_alexnet_features.npz')
VIT_IMAGE_FEATURES = Path(f'data/amazon/{DATASET}_vit_features.npz')
CLIP_IMAGE_FEATURES = Path(f'data/amazon/{DATASET}_clipimage_features.npz')

CLIP_TEXT_FEATURES = Path(f'data/amazon/{DATASET}_cliptext_features.npz')
BERT_TEXT_FEATURES = Path(f'data/amazon/{DATASET}_bert_features.npz')
DEST_FOLDER = Path(f'data/{DATASET}')

#assert ALEXNET_IMAGE_FEATURES.exists()
assert VIT_IMAGE_FEATURES.exists(), f"{VIT_IMAGE_FEATURES} does not exist"
assert CLIP_IMAGE_FEATURES.exists(), f"{CLIP_IMAGE_FEATURES} does not exist"
assert BERT_TEXT_FEATURES.exists(), f"{BERT_TEXT_FEATURES} does not exist"
assert CLIP_TEXT_FEATURES.exists(), f"{CLIP_TEXT_FEATURES} does not exist"

image_features = VIT_IMAGE_FEATURES
text_featues = BERT_TEXT_FEATURES

DEST_FOLDER.mkdir(exist_ok=True)

with open(DEST_FOLDER / 'signature.txt', mode='w') as f:
    f.write(f'Image: {image_features.stem}\nText: {text_featues.stem}')

In [8]:
def generate_unique_ids(series: pd.Series) -> pd.Series:
    rng = np.random.default_rng(RANDOM_SEED)
    unique_ids = series.unique()
    return pd.Series(
        index=rng.permutation(unique_ids), 
        data=range(len(unique_ids))
    )

def df_stats(df: pd.DataFrame) -> str:
    n_items = len(df['asin'].unique())
    n_users = len(df['reviewerID'].unique())
    sparsity = 1. * len(df) / (n_users * n_items)
    return f'{n_users} users {n_items} items ratings: {len(df)}. Sparsity {sparsity * 100:.3f}%'

def save_numerized(
    reviews: pd.DataFrame, 
    uids: pd.Series, 
    product_ids: pd.Series,
    user_id_column: str,
    product_ids_column: str,
    columns: List[str],
    dest: Path
):
    """Save a Dataframe following userids and product_ids"""
    joined = reviews.join(uids.to_frame(user_id_column), on='reviewerID')
    assert joined[user_id_column].isna().sum() == 0
    
    joined = joined.join(product_ids.to_frame(product_ids_column), on='asin')
    assert joined[product_ids_column].isna().sum() == 0
    
    res = joined[columns]
    
    res.to_csv(dest, index=False)

    return res

def split_train_test_proportion(df: pd.DataFrame, test_prop=0.2):
    """
    Split the dataframe by reviewer and take exactly `test_prop` records
    for test and leave the rest for training
    """
    rng = np.random.default_rng(RANDOM_SEED)

    res = df.copy()
    res['rnd'] = rng.random(size=len(df))
    res['rnd_rank'] = res.groupby('reviewerID')['rnd'].rank(pct=True)
    condition = res['rnd_rank'] <= test_prop
    
    (_, train), (_, test) = res.groupby(condition)
    
    assert(len(set(train.index) & set(test.index)) == 0)

    return df.loc[train.index], df.loc[test.index]

In [7]:
reviews_df = amazon_dataset.reviews_df(DATASET)
reviews_df.sample(n=3)

,id,asin,reviewerID,reviewerName,overall,text,reviewTime,summary,verified,vote
202419,8633896,B00HNTOYQC,A3DYQ8GDKCXOMS,Connie Bailey,5.0,"As others have already noted, this movie is a ...",2018-04-25,Perfect,True,NaN
39449,2123345,B0002V7T5Q,A3BU007A8PLDF,deana J maughan,4.0,doesn't play on what i have,2017-04-11,Four Stars,True,NaN
9792,385380,6301930789,A33A7PHDZ7WZCA,possum,5.0,Very interesting movie! I heard about it from ...,2017-05-02,The Handmaid's Tale 1990 Film,True,6.0


# Generate Data for SEM-MacridVAE & DMRL

Some data stats

In [9]:
print(DATASET)
print(df_stats(reviews_df))

Movies_and_TV
21974 users 23958 items ratings: 216110. Sparsity 0.041%


Sort some users randomly

In [10]:
user_ids = generate_unique_ids(reviews_df['reviewerID'])
user_ids

A1MB7I0VC5L7P3        0
A2TPW076Q0EXLE        1
AT1PIN6WA88V9         2
A3NGCB9QQMM9SM        3
A19ELFMMKB83HC        4
                  ...  
A2O9FHBZ34B5U5    21969
ARX5VIQRPMX4B     21970
A2EQ95QDK0PQCG    21971
A1PV5TW2LP04YU    21972
ASPZIB8XL7WAJ     21973
Length: 21974, dtype: int64

Same with products

In [11]:
item_ids = generate_unique_ids(reviews_df['asin'])
item_ids

B000E6ES94        0
B000PC6YTY        1
6303934870        2
B0024FADA6        3
6305926794        4
              ...  
B00YPNIK5U    23953
B001AOC9G8    23954
B00SWFMCHO    23955
B00005JPI6    23956
B000UJ48QS    23957
Length: 23958, dtype: int64

Split train test and validation by user

In [153]:
train_reviews, validation_test_reviews = split_train_test_proportion(reviews_df, test_prop=0.4)
validation_reviews, test_reviews = split_train_test_proportion(validation_test_reviews, test_prop=0.5)

In [129]:
len(train_reviews)

113836

len(validation_reviews)

In [130]:
len(test_reviews)

30728

Sanity checks. All users are in the train validation and test sets!

In [131]:
set(train_reviews['reviewerID']) == set(test_reviews['reviewerID']) ==  set(validation_reviews['reviewerID'])

True

Check that the train set and the validation set all have at least one item (same check as above)

In [132]:
pd.concat([
    train_reviews.groupby('reviewerID').size().to_frame('train_size'),
    validation_reviews.groupby('reviewerID').size().to_frame('val_size'),
    test_reviews.groupby('reviewerID').size().to_frame('test_size')
], axis=1).describe()

,train_size,val_size,test_size
count,23318.000000,23318.000000,23318.000000
mean,4.881894,1.474397,1.317780
std,3.529635,1.185903,1.091225
min,3.000000,1.000000,1.000000
25%,3.000000,1.000000,1.000000
50%,4.000000,1.000000,1.000000
75%,5.000000,2.000000,1.000000
max,61.000000,20.000000,20.000000


Generate users

In [133]:
user_ids.to_csv(DEST_FOLDER / 'users.txt')
item_ids.to_csv(DEST_FOLDER / 'items.txt', header=['item_id'])

In [134]:
NUMERIZED_COMMON_PARAMS = {
    'uids': user_ids, 
    'product_ids': item_ids,
    'user_id_column': 'user',
    'product_ids_column': 'item',
    'columns': ['user', 'item']
}

save_numerized(
    train_reviews, 
    dest=DEST_FOLDER / 'train.txt',
    **NUMERIZED_COMMON_PARAMS
)

,user,item
0,14891,30409
4,19073,29339
6,21394,38246
10,22275,38246
14,6565,38246
...,...,...
178934,5195,6878
178935,418,7919
178937,11611,5762
178938,16539,5762


In [135]:
save_numerized(
    validation_reviews, 
    dest=DEST_FOLDER / 'validation.txt',
    **NUMERIZED_COMMON_PARAMS
)
'Saved validation data'

'Saved validation data'

In [136]:
save_numerized(
    test_reviews, 
    dest=DEST_FOLDER / 'test.txt',
    **NUMERIZED_COMMON_PARAMS
)
'Saved test data'

'Saved test data'

In [137]:
def copy_features(
        item_ids: pd.Series, 
        features_file: Path
):
    print('Opening file')
    with np.load(features_file) as features:
        some_embedding = next(iter(features.values()))
        embedding_shape, = some_embedding.shape
        array_shape = (len(item_ids),  embedding_shape)
        print(f'Initializing array {array_shape}')
        res = np.full(array_shape,  fill_value=np.nan)

        for asin, idx in tqdm(item_ids.items(), 
                total=len(item_ids), unit_scale=True, unit='items', 
                desc='Copying features'):
            value = features.get(asin)
            if value is None:
                print(f'Item {asin} not found in features file')
                res[idx, :] = 0    
            else:
                assert np.isnan(value).sum() == 0, "Feature has NaN Values"
                res[idx, :] = value

        return res


image_features_array = copy_features(
    item_ids=item_ids, 
    features_file=image_features
)
image_features_array.shape

Opening file
Initializing array (38493, 1024)


Copying features: 100%|██████████| 38.5k/38.5k [01:21<00:00, 471items/s]


(38493, 1024)

In [138]:
IMAGE_EMBED_DEST = DEST_FOLDER / 'embed_image.npy'
np.save(IMAGE_EMBED_DEST, image_features_array)

print(f'{str(IMAGE_EMBED_DEST)}: {IMAGE_EMBED_DEST.stat().st_size // 2**20}MiB')

data/Clothing_Shoes_and_Jewelry/embed_image.npy: 300MiB


In [139]:
text_features_array = copy_features(
    item_ids=item_ids, 
    features_file=text_featues
)
text_features_array.shape

Opening file
Initializing array (38493, 1024)


Copying features: 100%|██████████| 38.5k/38.5k [01:28<00:00, 433items/s]


(38493, 1024)

In [140]:
TEXT_EMBED_DEST = DEST_FOLDER / 'embed_text.npy'
np.save(TEXT_EMBED_DEST, text_features_array)

print(f'{str(TEXT_EMBED_DEST)}: {TEXT_EMBED_DEST.stat().st_size // 2**20}MiB')

data/Clothing_Shoes_and_Jewelry/embed_text.npy: 300MiB


# Generate images

In [5]:
products_df = amazon_dataset.products_df(DATASET)
products_images_dir = amazon_dataset.product_images_dir(DATASET)
products_images_dir

PosixPath('data/amazon/Movies_and_TV_product_images')

This is necessary only for generating the fancy graph. Puts images on an transparent background

In [12]:
def convert_image(src: str, dst: str):
    min_white = np.array([235, 235, 235], np.uint8)
    white = np.array([255, 255, 255], np.uint8)
    img = cv2.imread(src)
    
    mask = 255 - cv2.inRange(img, min_white, white)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    res = cv2.bitwise_and(img, img, mask=mask)
    cv2.imwrite(dst, res)

def save_images(products_df: pd.DataFrame, item_ids: pd.Series, dest: Path):
    products_by_asin = products_df.set_index('asin')
    joined  = item_ids.to_frame('item_id').join(
        products_by_asin, validate='one_to_one'
    )
    dest.mkdir(exist_ok=True)
    
    no_images = 0
    
    progress = tqdm(
        joined.itertuples(), 
        desc='Copying images', 
        total=len(joined), 
        unit_scale=True, unit='items'
    )
    for row in progress:
        if not isinstance(row.image_slug, list):
            print('Not a list')
            continue
        if len(row.image_slug) == 0:
            no_images += 1
            progress.set_postfix_str(f'No images: {no_images}', refresh=False)
            continue
        slug = row.image_slug[0]
        src = products_images_dir / f"{slug}.jpg"
        assert src.exists()
        dst = dest / f'{row.item_id}.png'
        convert_image(str(src), str(dst))

#save_images(products_df, item_ids,  DEST_FOLDER / 'images')

Copying images:   0%|          | 42.0/24.0k [00:00<03:50, 104items/s, No images: 4] 

Not a list


Copying images:   2%|▏         | 541/24.0k [00:05<03:54, 99.9items/s, No images: 19]

Not a list


Copying images:   6%|▋         | 1.52k/24.0k [00:16<03:58, 94.0items/s, No images: 41]

...

Copying images:  93%|█████████▎| 22.2k/24.0k [03:35<00:16, 105items/s, No images: 561] 

Not a list


Copying images:  97%|█████████▋| 23.2k/24.0k [03:45<00:07, 106items/s, No images: 588] 

Not a list


Copying images: 100%|██████████| 24.0k/24.0k [03:52<00:00, 103items/s, No images: 607] 


In [15]:
def save_titles(products_df: pd.DataFrame, item_ids: pd.Series, dest: Path):
    products_by_asin = products_df.set_index('asin')
    joined  = item_ids.to_frame('item_id').join(
        products_by_asin, validate='one_to_one'
    )
    dest / 'item_titles.txt'
    joined[['item_id', 'title']].to_csv(dest / 'item_titles.txt', index=None)

# Not really needed for anything yet
#save_titles(products_df, item_ids,  DEST_FOLDER)

# Generate categories

In [143]:
# categories_df = amazon_dataset.product_categories_df(DATASET)
# products_df = amazon_dataset.products_df(DATASET)

In [144]:
# categories_id = generate_unique_ids(categories_df['name'])
# categories_id.to_csv(DEST_FOLDER / 'categories.txt', header=['category_id'])
# categories_id

In [145]:
def build_categories_matrix(
        categories_df: pd.DataFrame, 
        products_df: pd.DataFrame, 
        item_ids: pd.Series, 
        categories_id: pd.Series
    ):
    MIN_SUPPORT = 0.01
    MAX_SUPPORT = 0.9
    
    # Categories only with not too many or too few items
    categories_support = categories_df['name'].value_counts() 

    allowed_categories_names = categories_support[1:8].index

    print("How many allowed categories")
    print(len(allowed_categories_names))
    print(allowed_categories_names)
    
    filtered_categories = categories_df.loc[
        categories_df['name'].isin(allowed_categories_names)
    ]

    products_df = products_df.join(
        item_ids.to_frame('item_id'), on='asin', validate='1:1')

    res = filtered_categories.set_index('product_id').join(
        products_df, validate='m:1')
    res = res.reset_index(drop=True)
    
    # Check all categories are in categories_id
    res = res.join(categories_id.to_frame('category_id'), on='name')
    assert res['category_id'].isna().sum() == 0

    items_with_no_category = set(item_ids) - set(res['item_id'].unique())
    print(f"items with no category: {len(items_with_no_category)}")
    
    return products_df.loc[
        products_df['asin'].isin(item_ids[items_with_no_category].index)
    ]

    return res[['item_id', 'category_id']]

# categories_matrix = build_categories_matrix(
#     categories_df, products_df, item_ids, categories_id
# )
#categories_matrix
#categories_matrix.to_csv(DEST_FOLDER / 'categorical.txt', index=False)